In [1]:
import os

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder
from helper_funcs import *

import pandas

import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

from params import lsh_test, hypercube_test, kmeans_test, gnn_test, mrng_test, nsg_test, get_aaf

2023-12-31 02:00:30.476603: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-31 02:00:30.909116: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-31 02:00:30.909952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 02:00:31.932991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/thantrisp/miniconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'
    decoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_dataset.dat'
    decoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_decoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query,
                                        decoded_dataset, decoded_query]})

n = 60000

In [3]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # decode encoded datasets
    decoded_train = autoencoder.decode(encoded_train)
    decoded_test = autoencoder.decode(encoded_test)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

    # normalize decoded datasets
    decoded_train = normalize(decoded_train)
    decoded_test = normalize(decoded_test)

    # save decoded datasets
    save_decoded_binary(decoded_train, decoded_dataset)
    save_decoded_binary(decoded_test, decoded_query)

313/313 [==============================] - 1s 2ms/step


In [8]:
def objective_lsh(trial):
    model = trial.suggest_categorical('model', model_to_files.keys())
    param_dict = {'k': trial.suggest_int('k', 1, 10),
                  'L': trial.suggest_int('L', 1, 10),
                  'table_size':  trial.suggest_categorical('table_size', [int(n/32), int(n/16), int(n/8)]),
                  'window_size': trial.suggest_float('window_size', 0.01, 1),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False])
                 }
    
    print("Trial parameters:", param_dict)

    encoded_dataset, encoded_query = model_to_files[model][2:4]

    average_time, aaf_latent, min_neighbors = lsh_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, N=60, int_data=0)

    # trial should return at least 60 neighbors to be used in GNNS
    # penalize model if slower than brute force
    c0 = - min_neighbors.value + 60
    c1 = average_time.value - 1e-4
    trial.set_user_attr('constraint', (c0, c1))

    return aaf_latent.value, average_time.value

def constraints(trial):
    return trial.user_attrs['constraint']

In [9]:
%%time
sampler = optuna.samplers.NSGAIISampler(constraints_func=constraints)
lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'], sampler=sampler)
lsh_study.optimize(objective_lsh, n_trials=200, n_jobs=-1)
print("-------------------- Best trials --------------------")
trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
# print feasible trials only
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}, Constraints = {}".format(trial.values, trial.user_attrs["constraint"]))
    print(" Params = {}".format(trial.params))

/home/thantrisp/miniconda3/envs/tf/lib/python3.10/site-packages/optuna/samplers/nsgaii/_sampler.py:162: ExperimentalWarning: The constraints_func option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-12-31 02:05:13,491] A new study created in memory with name: lsh


Trial parameters: {'k': 7, 'L': 2, 'table_size': 3750, 'window_size': 0.3642489778595853, 'query_trick': True}
Trial parameters: {'k': 3, 'L': 10, 'table_size': 3750, 'window_size': 0.921906172240586, 'query_trick': False}
Trial parameters: {'k': 3, 'L': 9, 'table_size': 7500, 'window_size': 0.9291969327180418, 'query_trick': True}
Trial parameters: {'k': 9, 'L': 3, 'table_size': 1875, 'window_size': 0.06864599301500264, 'query_trick': True}
Trial parameters: {'k': 1, 'L': 2, 'table_size': 7500, 'window_size': 0.07386921385738791, 'query_trick': False}
Trial parameters: {'k': 3, 'L': 6, 'table_size': 1875, 'window_size': 0.5549995696512596, 'query_trick': True}
Trial parameters: {'k': 4, 'L': 1, 'table_size': 7500, 'window_size': 0.7563206209082894, 'query_trick': False}
Trial parameters: {'k': 4, 'L': 8, 'table_size': 3750, 'window_size': 0.01665697822524154, 'query_trick': False}
Trial parameters: {'k': 10, 'L': 9, 'table_size': 1875, 'window_size': 0.8581827429284906, 'query_trick':

[I 2023-12-31 02:05:16,620] Trial 10 finished with values: [1.8026045942309699, 0.005449880000000001] and parameters: {'model': 'model_dense_26.keras', 'k': 1, 'L': 5, 'table_size': 1875, 'window_size': 0.6300541519338538, 'query_trick': True}. 


Trial parameters: {'k': 2, 'L': 7, 'table_size': 1875, 'window_size': 0.2063402919074834, 'query_trick': False}


[W 2023-12-31 02:05:17,043] Trial 0 failed with parameters: {'model': 'model_conv_12.keras', 'k': 7, 'L': 2, 'table_size': 3750, 'window_size': 0.3642489778595853, 'query_trick': True} because of the following error: The value nan is not acceptable.
[W 2023-12-31 02:05:17,046] Trial 0 failed with value (nan, 0.00059396).


...

Trial parameters: {'k': 6, 'L': 10, 'table_size': 3750, 'window_size': 0.5753401837316775, 'query_trick': False}
Trial parameters: {'k': 4, 'L': 9, 'table_size': 3750, 'window_size': 0.9152206630716463, 'query_trick': False}


[I 2023-12-31 02:05:23,612] Trial 15 finished with values: [2.4697083001142337, 0.0016131700000000001] and parameters: {'model': 'model_dense_26.keras', 'k': 3, 'L': 6, 'table_size': 3750, 'window_size': 0.7811122443482683, 'query_trick': False}. 
[W 2023-12-31 02:05:23,693] Trial 17 failed with parameters: {'model': 'model_dense_43.keras', 'k': 4, 'L': 1, 'table_size': 3750, 'window_size': 0.9935626525156868, 'query_trick': True} because of the following error: The value nan is not acceptable.
[W 2023-12-31 02:05:23,695] Trial 17 failed with value (nan, 0.0006651700000000001).
[I 2023-12-31 02:05:23,818] Trial 18 finished with values: [1.4192101209693941, 0.011369740000000003] and parameters: {'model': 'model_conv_12.keras', 'k': 1, 'L': 9, 'table_size': 1875, 'window_size': 0.43308129313260163, 'query_trick': False}. 


Trial parameters: {'k': 10, 'L': 8, 'table_size': 3750, 'window_size': 0.36651313690900494, 'query_trick': False}
Trial parameters: {'k': 7, 'L': 5, 'table_size': 1875, 'window_size': 0.7250122164195588, 'query_trick': False}
Trial parameters: {'k': 5, 'L': 8, 'table_size': 7500, 'window_size': 0.5900457355812967, 'query_trick': True}


[W 2023-12-31 02:05:24,805] Trial 16 failed with parameters: {'model': 'model_dense_43.keras', 'k': 6, 'L': 5, 'table_size': 3750, 'window_size': 0.3271085435565295, 'query_trick': True} because of the following error: The value nan is not acceptable.
[W 2023-12-31 02:05:24,812] Trial 16 failed with value (nan, 0.0013670499999999999).
[I 2023-12-31 02:05:25,528] Trial 19 finished with values: [1.898380214943132, 0.003885969999999999] and parameters: {'model': 'model_conv_46.keras', 'k': 7, 'L': 10, 'table_size': 3750, 'window_size': 0.607907243156738, 'query_trick': False}. 
[I 2023-12-31 02:05:25,721] Trial 20 finished with values: [1.8388522335829887, 0.0016469199999999994] and parameters: {'model': 'model_conv_12.keras', 'k': 7, 'L': 9, 'table_size': 7500, 'window_size': 0.5982639536997318, 'query_trick': False}. 
[W 2023-12-31 02:05:25,820] Trial 22 failed with parameters: {'model': 'model_dense_43.keras', 'k': 4, 'L': 4, 'table_size': 1875, 'window_size': 0.6389137671436489, 'quer

KeyboardInterrupt: 

In [ ]:
df = lsh_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['value'])
df_sorted = df_sorted.sort_values(by=['value'], ascending=True)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

In [ ]:
plot_pareto_front(lsh_study, target_names=['aaf', 'average_time'])

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')